In [7]:
import gc
import pandas as pd
import numpy as np
import lightgbm as lgb
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.multioutput import MultiOutputRegressor, MultiOutputClassifier
from sklearn.metrics import log_loss

In [2]:
pd.set_option("max_columns", 300)
pd.set_option("max_rows", 300)

In [3]:
df = pd.read_pickle("../data/train_data/train.pkl")

In [4]:
print(df.shape)
df.head()

(1180312, 20)


,bakaze,kyoku_num,honba,kyotaku,dora,parent,riichi_player,agari_hai,player0_tehai,player0_sutehai,player0_point,player1_tehai,player1_sutehai,player1_point,player2_tehai,player2_sutehai,player2_point,player3_tehai,player3_sutehai,player3_point
0,0,1,0,0,[24],0,0,"[12, 15]","[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[9, 1, 11, 16, 5]",250,"[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[9, 1, 11, 16, 5]",250,"[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[9, 1, 11, 16, 5]",250,"[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[9, 1, 11, 16, 5]",250
1,0,1,1,0,[26],0,0,[9],"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, ...","[33, 19, 32, 34, 35, 12, 4, 14, 7]",308,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, ...","[33, 19, 32, 34, 35, 12, 4, 14, 7]",308,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, ...","[33, 19, 32, 34, 35, 12, 4, 14, 7]",308,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, ...","[33, 19, 32, 34, 35, 12, 4, 14, 7]",308
2,0,1,2,0,[31],0,2,[7],"[0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 0, 0, 0, 0, 0, ...","[34, 31, 36, 35, 33, 1, 21, 29, 33, 11, 13, 19...",171,"[0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, ...","[21, 37, 9, 37, 32, 31, 28, 3, 9, 33, 32, 8, 16]",381,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, ...","[27, 1, 1, 37, 6, 2, 7, 6, 28, 23, 33, 5, 29]",219,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, ...","[27, 1, 1, 37, 6, 2, 7, 6, 28, 23, 33, 5, 29]",219
3,0,2,0,0,[32],1,3,[28],"[0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, ...","[34, 21, 22, 12, 35, 12, 4, 15]",197,"[0, 1, 1, 1, 1, 2, 1, 0, 2, 0, 0, 0, 0, 0, 0, ...","[29, 34, 37, 35, 36, 9, 33]",197,"[0, 1, 1, 1, 1, 2, 1, 0, 2, 0, 0, 0, 0, 0, 0, ...","[29, 34, 37, 35, 36, 9, 33]",197,"[0, 1, 1, 1, 1, 2, 1, 0, 2, 0, 0, 0, 0, 0, 0, ...","[29, 34, 37, 35, 36, 9, 33]",197
4,0,3,0,0,[1],2,1,"[1, 4]","[0, 0, 1, 1, 0, 2, 1, 2, 2, 1, 0, 0, 0, 0, 0, ...","[32, 34, 33, 35, 32, 23, 36, 26, 14, 12, 29, 29]",117,"[0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ...","[26, 21, 11, 13, 9, 12, 15, 36, 24, 33, 4, 13]",340,"[0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ...","[26, 21, 11, 13, 9, 12, 15, 36, 24, 33, 4, 13]",340,"[0, 0, 1, 1, 0, 2, 1, 2, 2, 1, 0, 0, 0, 0, 0, ...","[32, 34, 33, 35, 32, 23, 36, 26, 14, 12, 29, 29]",117


In [5]:
def func_33_to_40(x):
    if x < 9:
        return x+1
    elif x < 18:
        return x+2
    elif x < 27:
        return x+3
    else:
        return x+4
df["dora"] = df["dora"].map(lambda h_list: [func_33_to_40(h//4) for h in h_list])

In [6]:
# メモリ削減のためplayer1以外の手牌のカラムは消す
df.drop(["player0_tehai", "player2_tehai", "player3_tehai"], axis=1, inplace=True)

In [8]:
gc.collect()

60

## make feature

In [9]:
def make_feature1(df):
    # ドラの数
    df["dora_num"] = df["dora"].map(len)
    # 捨て牌の数
    df["sutehai_num"] = df["player0_sutehai"].map(len)
    # 捨て牌の字牌率
    df["sutehai_jihai_rate"] = df["player0_sutehai"].map(lambda h_list: np.mean([h > 30 for h in h_list]))
    # 捨て牌の19字牌率
    df["sutehai_yaochu_rate"] = df["player0_sutehai"].map(
        lambda h_list: np.mean([h > 30 or h%10 ==1 or h%10 ==9 for h in h_list]))
    # 捨て牌の456牌率
    df["sutehai_456_rate"] = df["player0_sutehai"].map(
        lambda h_list: np.mean([h%10 in [4, 5, 6] for h in h_list]))
    # 宣言牌
    sengen_hai = df["player0_sutehai"].map(lambda x: x[-1])
    # 宣言牌の数字（字牌は-1）
    df["sengen_hai"] = sengen_hai
    df["sengen_hai_number"] = sengen_hai.map(lambda x: x%10 if x < 30 else -1)
    df["sengen_hai_color"] = sengen_hai.map(lambda x: x//10)
    df["sengen_hai_count"] = sengen_hai.map(sengen_hai.value_counts(normalize=True))
    return df

In [10]:
df = make_feature1(df)

In [11]:
def contain_dora(df):
    # 最初のドラが捨て牌に含まれているか
    df["idx"] = df.index
    dora_dict = df["dora"].map(lambda x: x[0]).to_dict()
    sutehai_dict = df["player0_sutehai"].to_dict()
    def contain_func(idx):
        dora = dora_dict[idx]
        sutehai = sutehai_dict[idx]
        return int(dora in set(sutehai))
    dora_in_sutehai = df["idx"].map(contain_func)
    del df["idx"]
    return dora_in_sutehai

In [12]:
def make_feature2(df):
    # 捨て牌にドラ(1つ目)が含まれているか(0 or 1)
    df["sutehai_dora_sum"] = contain_dora(df)
    return df

In [13]:
df = make_feature2(df)

In [14]:
def make_feature3(df):
    # 序盤(~3, ~6)にヤオチュウ牌以外を切っている割合
    df["ratio_not_yaochu_in_first3"] = df["player0_sutehai"].map(
        lambda h_list: np.mean([h < 30 and h%10 in (2, 3, 4, 5, 6, 7, 8) for h in h_list[:3]]))
    df["ratio_not_yaochu_in_first6"] = df["player0_sutehai"].map(
        lambda h_list: np.mean([h < 30 and h%10 in (2, 3, 4, 5, 6, 7, 8) for h in h_list[:6]]))
    # 序盤(~3, ~6)に4, 5, 6を切っている割合
    df["ratio_456_in_first3"] = df["player0_sutehai"].map(
        lambda h_list: np.mean([h < 30 and h%10 in (4, 5, 6) for h in h_list[:3]]))
    df["ratio_456_in_first6"] = df["player0_sutehai"].map(
        lambda h_list: np.mean([h < 30 and h%10 in (4, 5,  6) for h in h_list[:6]]))
    return df

In [15]:
df = make_feature3(df)

In [16]:
def make_feature4(df):
    # 捨て牌で各牌を何枚ずつ切ったか
    sutehai_count = df["player0_sutehai"].apply(lambda x: pd.Series(np.eye(40)[x].sum(axis=0)))
    use_col = [c for c in range(40) if c < 38 and c%10 != 0]
    sutehai_count = sutehai_count.loc[:, use_col]
    sutehai_count.columns = [f"discard_{c}" for c in use_col]
    df = pd.concat([df, sutehai_count], axis=1)
    return df

In [17]:
df = make_feature4(df)

In [18]:
def make_feature5(df):
    # 各牌が場に何枚見えているか
    # 各プレイヤーの捨て牌
    sutehai_count0 = df["player0_sutehai"].apply(lambda x: pd.Series(np.eye(40)[x].sum(axis=0)))
    sutehai_count1 = df["player1_sutehai"].apply(lambda x: pd.Series(np.eye(40)[x].sum(axis=0)))
    sutehai_count2 = df["player2_sutehai"].apply(lambda x: pd.Series(np.eye(40)[x].sum(axis=0)))
    sutehai_count3 = df["player3_sutehai"].apply(lambda x: pd.Series(np.eye(40)[x].sum(axis=0)))
    sutehai_count_all = sutehai_count0 + sutehai_count1 + sutehai_count2 + sutehai_count3
    # プレイヤー1目線として、プレイヤー1の手牌
    tehai_count1 = df["player1_tehai"].apply(lambda x: pd.Series(np.eye(40)[x].sum(axis=0)))
    # ドラ表示牌
    def pre_dora_func(x):
        if x < 30:
            if x%10 > 1:
                return x-1
            else:
                return x//10 + 9
        else:
            if x == 31:
                return 34
            if x == 35:
                return 37
            else:
                return x-1                
            
    pre_dora = df["dora"].map(
        lambda h_list: [pre_dora_func(h) for h in h_list]).apply(lambda x: pd.Series(np.eye(40)[x].sum(axis=0)))
    # sum
    count_all = sutehai_count_all + tehai_count1
    use_col = [c for c in range(40) if c < 38 and c%10 != 0]
    count_all = count_all.loc[:, use_col]
    count_all.columns = [f"can_see_count_{c}" for c in use_col]
    df = pd.concat([df, count_all], axis=1)
    return df

In [19]:
df = make_feature5(df)

In [21]:
df.head()

,bakaze,kyoku_num,honba,kyotaku,dora,parent,riichi_player,agari_hai,player0_sutehai,player0_point,player1_tehai,player1_sutehai,player1_point,player2_sutehai,player2_point,player3_sutehai,player3_point,dora_num,sutehai_num,sutehai_jihai_rate,sutehai_yaochu_rate,sutehai_456_rate,sengen_hai,sengen_hai_number,sengen_hai_color,sengen_hai_count,sutehai_dora_sum,ratio_not_yaochu_in_first3,ratio_not_yaochu_in_first6,ratio_456_in_first3,ratio_456_in_first6,discard_1,discard_2,discard_3,discard_4,discard_5,discard_6,discard_7,discard_8,discard_9,discard_11,discard_12,discard_13,discard_14,discard_15,discard_16,discard_17,discard_18,discard_19,discard_21,discard_22,discard_23,discard_24,discard_25,discard_26,discard_27,discard_28,discard_29,discard_31,discard_32,discard_33,discard_34,discard_35,discard_36,discard_37,can_see_count_1,can_see_count_2,can_see_count_3,can_see_count_4,can_see_count_5,can_see_count_6,can_see_count_7,can_see_count_8,can_see_count_9,can_see_count_11,can_see_count_12,can_see_count_13,can_see_count_14,can_see_count_15,can_see_count_16,can_see_count_17,can_see_count_18,can_see_count_19,can_see_count_21,can_see_count_22,can_see_count_23,can_see_count_24,can_see_count_25,can_see_count_26,can_see_count_27,can_see_count_28,can_see_count_29,can_see_count_31,can_see_count_32,can_see_count_33,can_see_count_34,can_see_count_35,can_see_count_36,can_see_count_37
0,0,1,0,0,[7],0,0,"[12, 15]","[9, 1, 11, 16, 5]",250,"[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[9, 1, 11, 16, 5]",250,"[9, 1, 11, 16, 5]",250,"[9, 1, 11, 16, 5]",250,1,5,0.000000,0.600000,0.400000,5,5,0,0.041209,0,0.000000,0.400000,0.0,0.4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,1,0,[7],0,0,[9],"[33, 19, 32, 34, 35, 12, 4, 14, 7]",308,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, ...","[33, 19, 32, 34, 35, 12, 4, 14, 7]",308,"[33, 19, 32, 34, 35, 12, 4, 14, 7]",308,"[33, 19, 32, 34, 35, 12, 4, 14, 7]",308,1,9,0.444444,0.555556,0.444444,7,7,0,0.037592,1,0.000000,0.166667,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,10.0,0.0,1.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,4.0,4.0,0.0,0.0
2,0,1,2,0,[8],0,2,[7],"[34, 31, 36, 35, 33, 1, 21, 29, 33, 11, 13, 19...",171,"[0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, ...","[21, 37, 9, 37, 32, 31, 28, 3, 9, 33, 32, 8, 16]",381,"[27, 1, 1, 37, 6, 2, 7, 6, 28, 23, 33, 5, 29]",219,"[27, 1, 1, 37, 6, 2, 7, 6, 28, 23, 33, 5, 29]",219,1,13,0.461538,0.846154,0.307692,24,4,2,0.034289,0,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,1.0,1.0,1.0,0.0,16.0,3.0,1.0,0.0,2.0,4.0,2.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,2.0,1.0,0.0,0.0,2.0,3.0,3.0,2.0,2.0,5.0,1.0,1.0,1.0,4.0
3,0,2,0,0,[9],1,3,[28],"[34, 21, 22, 12, 35, 12, 4, 15]",197,"[0, 1, 1, 1, 1, 2, 1, 0, 2, 0, 0, 0, 0, 0, 0, ...","[29, 34, 37, 35, 36, 9, 33]",197,"[29, 34, 37, 35, 36, 9, 33]",197,"[29, 34, 37, 35, 36, 9, 33]",197,1,8,0.250000,0.375000,0.500000,15,5,1,0.041277,0,0.333333,0.500000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,9.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,4.0,4.0,3.0,3.0
4,0,3,0,0,[1],2,1,"[1, 4]","[32, 34, 33, 35, 32, 23, 36, 26, 14, 12, 29, 29]",117,"[0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ...","[26, 21, 11, 13, 9, 12, 15, 36, 24, 33, 4, 13]",340,"[26, 21, 11, 13, 9, 12, 15, 36, 24, 33, 4, 13]",340,"[32, 34, 33, 35, 32, 23, 36, 26, 14, 12, 29, 29]",117,1,12,0.50

In [23]:
df.to_pickle("../data/feature/lgb_test.pkl")

In [24]:
train_df = df[df.agari_hai.map(len) > 0].reset_index(drop=True)

In [25]:
train_df.head()

,bakaze,kyoku_num,honba,kyotaku,dora,parent,riichi_player,agari_hai,player0_sutehai,player0_point,player1_tehai,player1_sutehai,player1_point,player2_sutehai,player2_point,player3_sutehai,player3_point,dora_num,sutehai_num,sutehai_jihai_rate,sutehai_yaochu_rate,sutehai_456_rate,sengen_hai,sengen_hai_number,sengen_hai_color,sengen_hai_count,sutehai_dora_sum,ratio_not_yaochu_in_first3,ratio_not_yaochu_in_first6,ratio_456_in_first3,ratio_456_in_first6,discard_1,discard_2,discard_3,discard_4,discard_5,discard_6,discard_7,discard_8,discard_9,discard_11,discard_12,discard_13,discard_14,discard_15,discard_16,discard_17,discard_18,discard_19,discard_21,discard_22,discard_23,discard_24,discard_25,discard_26,discard_27,discard_28,discard_29,discard_31,discard_32,discard_33,discard_34,discard_35,discard_36,discard_37,can_see_count_1,can_see_count_2,can_see_count_3,can_see_count_4,can_see_count_5,can_see_count_6,can_see_count_7,can_see_count_8,can_see_count_9,can_see_count_11,can_see_count_12,can_see_count_13,can_see_count_14,can_see_count_15,can_see_count_16,can_see_count_17,can_see_count_18,can_see_count_19,can_see_count_21,can_see_count_22,can_see_count_23,can_see_count_24,can_see_count_25,can_see_count_26,can_see_count_27,can_see_count_28,can_see_count_29,can_see_count_31,can_see_count_32,can_see_count_33,can_see_count_34,can_see_count_35,can_see_count_36,can_see_count_37
0,0,1,0,0,[7],0,0,"[12, 15]","[9, 1, 11, 16, 5]",250,"[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, ...","[9, 1, 11, 16, 5]",250,"[9, 1, 11, 16, 5]",250,"[9, 1, 11, 16, 5]",250,1,5,0.000000,0.600000,0.400000,5,5,0,0.041209,0,0.000000,0.400000,0.0,0.4,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,0.0,1.0,0.0,4.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,1,1,0,[7],0,0,[9],"[33, 19, 32, 34, 35, 12, 4, 14, 7]",308,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, ...","[33, 19, 32, 34, 35, 12, 4, 14, 7]",308,"[33, 19, 32, 34, 35, 12, 4, 14, 7]",308,"[33, 19, 32, 34, 35, 12, 4, 14, 7]",308,1,9,0.444444,0.555556,0.444444,7,7,0,0.037592,1,0.000000,0.166667,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,10.0,0.0,1.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,4.0,4.0,0.0,0.0
2,0,1,2,0,[8],0,2,[7],"[34, 31, 36, 35, 33, 1, 21, 29, 33, 11, 13, 19...",171,"[0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, ...","[21, 37, 9, 37, 32, 31, 28, 3, 9, 33, 32, 8, 16]",381,"[27, 1, 1, 37, 6, 2, 7, 6, 28, 23, 33, 5, 29]",219,"[27, 1, 1, 37, 6, 2, 7, 6, 28, 23, 33, 5, 29]",219,1,13,0.461538,0.846154,0.307692,24,4,2,0.034289,0,0.000000,0.000000,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,1.0,1.0,1.0,0.0,16.0,3.0,1.0,0.0,2.0,4.0,2.0,1.0,2.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,2.0,1.0,0.0,0.0,2.0,3.0,3.0,2.0,2.0,5.0,1.0,1.0,1.0,4.0
3,0,2,0,0,[9],1,3,[28],"[34, 21, 22, 12, 35, 12, 4, 15]",197,"[0, 1, 1, 1, 1, 2, 1, 0, 2, 0, 0, 0, 0, 0, 0, ...","[29, 34, 37, 35, 36, 9, 33]",197,"[29, 34, 37, 35, 36, 9, 33]",197,"[29, 34, 37, 35, 36, 9, 33]",197,1,8,0.250000,0.375000,0.500000,15,5,1,0.041277,0,0.333333,0.500000,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,9.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,3.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,3.0,4.0,4.0,3.0,3.0
4,0,3,0,0,[1],2,1,"[1, 4]","[32, 34, 33, 35, 32, 23, 36, 26, 14, 12, 29, 29]",117,"[0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, ...","[26, 21, 11, 13, 9, 12, 15, 36, 24, 33, 4, 13]",340,"[26, 21, 11, 13, 9, 12, 15, 36, 24, 33, 4, 13]",340,"[32, 34, 33, 35, 32, 23, 36, 26, 14, 12, 29, 29]",117,1,12,0.50

## Train

In [26]:
def make_ohe_target(y_srs):
    y_ohe = y_srs.apply(lambda x: pd.Series(np.eye(40)[x].sum(axis=0)))
    use_col = [c for c in range(40) if c < 38 and c%10 != 0]
    y_ohe = y_ohe.loc[:, use_col]
    return y_ohe

In [30]:
def lgb_train(train_df, ohe_targets):
    except_feats = [
        "dora",
        "agari_hai",
        "player0_tehai",
        "player1_tehai",
        "player2_tehai",
        "player3_tehai",
        "player0_sutehai",
        "player1_sutehai",
        "player2_sutehai",
        "player3_sutehai",
    ]
    use_feats = [c for c in train_df.columns if c not in except_feats]
    
    lgb_params = {
            'objective':'binary',
            "metric":"binary_logloss",
            "verbosity": -1,
            "boosting": "gbdt",
            'learning_rate': 0.05,
            'num_leaves': 32,
            'min_data_in_leaf': 30, 
            'max_depth': 4,
            "bagging_freq": 1,
            "bagging_fraction": 0.9,
            "bagging_seed": 11,
            "lambda_l1": 0.3,
            "lambda_l2": 0.3,
            "feature_fraction": 0.8,
            "seed": 11,
            "num_threads": 4,
            "max_bins": 30
    }
    oof_preds = np.zeros(ohe_targets.shape)
    importances = pd.DataFrame()
    folds = KFold(n_splits=5, random_state=2020)
    
    for i, (trn_idx, val_idx) in enumerate(folds.split(train_df)):
        print("="*20 + str(i) + "="*20)
        train_X = train_df.loc[trn_idx, use_feats].values
        train_y = ohe_targets.loc[trn_idx].values
        valid_X = train_df.loc[val_idx, use_feats].values
        valid_y = ohe_targets.loc[val_idx].values

        for hai_idx, hai_col in enumerate(ohe_targets.columns):
            trn_data = lgb.Dataset(
                train_X,
                label=train_y[:, hai_idx]
            )
            val_data = lgb.Dataset(
                valid_X,
                label=valid_y[:, hai_idx]
            )

            clf = lgb.train(
                        lgb_params,
                        trn_data,
                        5000,
                        valid_sets = [trn_data, val_data],
                        verbose_eval=1000,
                        early_stopping_rounds = 200,
            )
            imp_df = pd.DataFrame()
            imp_df['feature'] = use_feats
            imp_df['gain'] = clf.feature_importance(importance_type="gain")
            imp_df['fold'] = i + 1
            imp_df["hai"] = hai_col
            importances = pd.concat([importances, imp_df], axis=0, sort=False)

            oof_preds[val_idx, hai_idx] = clf.predict(valid_X)
            print(oof_preds.shape)
    return oof_preds, importances

In [28]:
ohe_targets = make_ohe_target(train_df["agari_hai"])

In [31]:
oof_preds, importances = lgb_train(train_df, ohe_targets)

====================0====================
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[414]	training's binary_logloss: 0.15861	valid_1's binary_logloss: 0.159678
(1126640, 34)
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[366]	training's binary_logloss: 0.199274	valid_1's binary_logloss: 0.202504
(1126640, 34)
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[542]	training's binary_logloss: 0.231302	valid_1's binary_logloss: 0.235383
(1126640, 34)
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[592]	training's binary_logloss: 0.270589	valid_1's binary_logloss: 0.272829
(1126640, 34)
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[476]	training's binary_logloss: 0.272788	valid_1's binary_logloss: 0.277025
(1126640, 34)
Training until vali

Early stopping, best iteration is:
[413]	training's binary_logloss: 0.16049	valid_1's binary_logloss: 0.163242
(1126640, 34)
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[493]	training's binary_logloss: 0.157082	valid_1's binary_logloss: 0.158962
(1126640, 34)
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[614]	training's binary_logloss: 0.197898	valid_1's binary_logloss: 0.199507
(1126640, 34)
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[701]	training's binary_logloss: 0.229849	valid_1's binary_logloss: 0.231958
(1126640, 34)
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[653]	training's binary_logloss: 0.267936	valid_1's binary_logloss: 0.271122
(1126640, 34)
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[639]	training's binary_lo

Early stopping, best iteration is:
[384]	training's binary_logloss: 0.159013	valid_1's binary_logloss: 0.159421
(1126640, 34)
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[412]	training's binary_logloss: 0.15712	valid_1's binary_logloss: 0.158418
(1126640, 34)
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[389]	training's binary_logloss: 0.197896	valid_1's binary_logloss: 0.198855
(1126640, 34)
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[654]	training's binary_logloss: 0.229801	valid_1's binary_logloss: 0.229537
(1126640, 34)
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[692]	training's binary_logloss: 0.267381	valid_1's binary_logloss: 0.269564
(1126640, 34)
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[486]	training's binary_lo

(1126640, 34)
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[559]	training's binary_logloss: 0.0411481	valid_1's binary_logloss: 0.0440522
(1126640, 34)
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[392]	training's binary_logloss: 0.0410515	valid_1's binary_logloss: 0.0421278
(1126640, 34)
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[244]	training's binary_logloss: 0.040484	valid_1's binary_logloss: 0.0421232
(1126640, 34)
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[431]	training's binary_logloss: 0.0393869	valid_1's binary_logloss: 0.0412063
(1126640, 34)
Training until validation scores don't improve for 200 rounds
Early stopping, best iteration is:
[426]	training's binary_logloss: 0.0434838	valid_1's binary_logloss: 0.0465676
(1126640, 34)
Training until validation scores don'

In [32]:
# データ数が1/5くらいのときは、5.731376468695772
log_loss(ohe_targets, oof_preds)

5.705283438316685

In [317]:
#　すべて0だった場合
log_loss(ohe_targets, np.zeros(ohe_targets.shape))

6.52906571424268

In [33]:
importances.groupby("feature")["gain"].mean().sort_values(ascending=False).reset_index().iloc[:10]

,feature,gain
0,sutehai_num,11585.214175
1,discard_2,8555.417119
2,discard_18,8157.026804
3,discard_22,8069.127762
4,discard_12,7949.266829
5,discard_24,7882.143176
6,discard_8,7869.265152
7,discard_26,7847.056172
8,discard_14,7845.393683
9,discard_29,7719.674432


In [34]:
importances[importances.hai==31].groupby("feature")["gain"].mean().sort_values(ascending=False).reset_index().iloc[:10]

,feature,gain
0,can_see_count_31,69342.316669
1,discard_31,26319.354210
2,riichi_player,13679.306203
3,can_see_count_2,13004.594240
4,can_see_count_1,3512.147801
5,sutehai_num,2506.788605
6,player0_point,2186.209017
7,sutehai_jihai_rate,2013.470067
8,sutehai_yaochu_rate,1798.069317
9,player1_point,1346.818059


## Save Results

In [35]:
importances.to_pickle("001_importance.pkl")

In [36]:
train_df.to_pickle("001_train_df.pkl")

In [37]:
pd.DataFrame(oof_preds).to_pickle("001_oof_preds.pkl")

In [38]:
def int_to_hai(x):
    jihai_dict = {
        31: "東",
        32: "南",
        33: "西",
        34: "北",
        35: "白",
        36: "発",
        37: "中"
    }
    if x < 10:
        return f"{x%10}m"
    elif x < 20:
        return f"{x%10}p"
    elif x < 30:
        return f"{x%10}s"
    else:
        return jihai_dict[x]

def show_sutehai(hai_list):
    print([int_to_hai(x) for x in hai_list[:6]])
    print([int_to_hai(x) for x in hai_list[6:12]])
    print([int_to_hai(x) for x in hai_list[12:]])

In [39]:
def show_pred(pred_arr):
    show_arr = pred_arr.round(4)
    display(
        pd.DataFrame(
            [show_arr[:9], show_arr[9:18], show_arr[18:27]],
            index = ["萬子", "ピンズ", "ソウズ",],
            columns = [i for i in range(1, 10)]
        )
    )
    display(
        pd.DataFrame(
            [show_arr[27:]],
            index = ["字牌"],
            columns = [int_to_hai(i)for i in range(31, 38)]
        )
    )

In [40]:
show_sutehai(train_df["player0_sutehai"].iloc[0])

['9m', '1m', '1p', '6p', '5m']
[]
[]


In [41]:
show_pred(oof_preds[0])

,1,2,3,4,5,6,7,8,9
萬子,0.0025,0.0314,0.0659,0.0507,0.0007,0.0558,0.0614,0.0324,0.0021
ピンズ,0.0032,0.0868,0.0396,0.0503,0.1115,0.0015,0.0437,0.0404,0.0129
ソウズ,0.0645,0.0904,0.1098,0.1319,0.1251,0.1365,0.1128,0.0955,0.0791


,東,南,西,北,白,発,中
字牌,0.0138,0.0169,0.0116,0.0118,0.0124,0.0154,0.0154
